In [ ]:
# 필요한 모듈 임포트
!pip install sacremoses
!pip install transformers
!pip install sentencepiece
import tensorflow as tf
import numpy as np
import pandas as pd
from transformers import *
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
import re
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

     |████████████████████████████████| 880 kB 5.4 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=145fe5bad241af905fc371345aad399dab798aaa784498a5aa7370f590c3ac9a
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
     |████████████████████████████████| 4.2 MB 5.3 MB/s 
     |████████████████████████████████| 596 kB 77.5 MB/s 
     |████████████████████████████████| 84 kB 2.9 MB/s 
     |████████████████████████████████| 6.6 MB 67.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.2 MB 5.3 MB/s 


In [ ]:
# 데이터 가져오기
# 이 데이터는 토큰화가 이미 완료되어 있음
dataset_folder_path = '/content/drive/MyDrive/Colab_Notebooks/2nd_project/dataset'
with open(f'{dataset_folder_path}/seq.in','r') as f:
    x_data = f.readlines()
with open(f'{dataset_folder_path}/seq.out','r') as f:
    y_data = f.readlines()

In [ ]:
train_x = []
train_y = []
for i in x_data:
  train_x.append(re.sub('\n','',i))
for j in y_data:
  train_y.append(re.sub('\n','',j))

In [59]:
train_x[33]

'상 큼 하며_ 와 인으로_ 무 겁 지_ 않은_ 느낌 을_ 주는_ 샴 페이 ᆫ _ 추 천_'

In [ ]:
train_y[33]

'산미 산미 산미 O O 바디감 바디감 바디감 바디감 O O O 종류 종류 종류 종류 O O'

In [ ]:
# 라벨을 숫자로 딕셔너리형태로 저장
labels = ['O', '금액', '당도', '바디감', '산미', '종류']
label_dict = {word:i for i, word in enumerate(labels)}
label_dict.update({"[PAD]":len(label_dict)})
index_to_ner = {i:j for j, i in label_dict.items()}

In [ ]:
# [토큰,라벨] 형태로 가공
data = []
tmp = []
for i,j in zip(train_x,train_y):
  tmp = []
  for a,b in zip(i.split(),j.split()):
    tmp.append([a,label_dict[b]])
  data.append(tmp)

In [ ]:
data[32:34]

[[['안_', 4],
  ['신_', 4],
  ['10', 1],
  ['만원_', 1],
  ['이하_', 1],
  ['취', 0],
  ['하지_', 0],
  ['않', 0],
  ['게_', 0],
  ['기', 0],
  ['분', 0],
  ['만_', 0],
  ['낼_', 0],
  ['수_', 0],
  ['있는_', 0],
  ['와', 0],
  ['인_', 0],
  ['추', 0],
  ['천', 0],
  ['해', 0],
  ['줘', 0],
  ['_', 0]],
 [['상', 4],
  ['큼', 4],
  ['하며_', 4],
  ['와', 0],
  ['인으로_', 0],
  ['무', 3],
  ['겁', 3],
  ['지_', 3],
  ['않은_', 3],
  ['느낌', 0],
  ['을_', 0],
  ['주는_', 0],
  ['샴', 5],
  ['페이', 5],
  ['ᆫ', 5],
  ['_', 5],
  ['추', 0],
  ['천_', 0]]]

In [ ]:
# 토큰 앞에 [CLS], 끝에 [SEP]추가 및 태그에 ['O']에 맞는 라벨 추가
sentences = []
targets = []
for tup in data:
  sentence = []
  target = []
  sentence.append("[CLS]")
  target.append(label_dict['O'])
  for i, j in tup:
    sentence.append(i)
    target.append(j)
  sentence.append("[SEP]")
  target.append(label_dict['O'])
  sentences.append(sentence)
  targets.append(target)

In [ ]:
sentences[33]

['[CLS]',
 '상',
 '큼',
 '하며_',
 '와',
 '인으로_',
 '무',
 '겁',
 '지_',
 '않은_',
 '느낌',
 '을_',
 '주는_',
 '샴',
 '페이',
 'ᆫ',
 '_',
 '추',
 '천_',
 '[SEP]']

In [ ]:
targets[33]

[0, 4, 4, 4, 0, 0, 3, 3, 3, 3, 0, 0, 0, 5, 5, 5, 5, 0, 0, 0]

In [ ]:
# 패딩을 위해 가장 긴 값 추출
print(max(np.array([len(x) for x in sentences])))
max_len = 41
bs = 32

41


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

loading file https://huggingface.co/bert-base-multilingual-cased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/eff018e45de5364a8368df1f2df3461d506e2a111e9dd50af1fae061cd460ead.6c5b6600e968f4b5e08c86d8891ea99e51537fc2bf251435fb46922e8f7a7b29
loading file https://huggingface.co/bert-base-multilingual-cased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-multilingual-cased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-multilingual-cased/resolve/main/tokenizer_config.json from cache at /root/.cache/huggingface/transformers/f55e7a2ad4f8d0fff2733b3f79777e1e99247f2e4583703e92ce74453af8c235.ec5c189f89475aac7d8cbd243960a0655cfadc3d0474da8ff2ed0bf1699c2a5f
loading configuration file https://huggingface.co/bert-base-multilingual-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/6c4a5d81a58c9791cdf76a09bce1b5abfb9cf958aebada51200f4515

In [ ]:
# input_ids : 문장이 토크나이즈 된 것이 숫자로 바뀐 것
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in sentences],
                          maxlen=max_len, dtype = "int", value=tokenizer.convert_tokens_to_ids("[PAD]"), truncating="post", padding="post")

In [ ]:
input_ids[33]

array([ 101, 9414, 9837,  100, 9590,  100, 9294, 8869,  100,  100,  100,
        100,  100,  100,  100,  100,  168, 9765,  100,  102,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0])

In [ ]:
# 패딩에 해당하는 부분은 label_dict([PAD]) (6)이 들어가게 되겠습니다.
tags = pad_sequences([lab for lab in targets], maxlen=max_len, value=label_dict["[PAD]"], padding='post',
                     dtype='int', truncating='post')

In [ ]:
tags[33]

array([0, 4, 4, 4, 0, 0, 3, 3, 3, 3, 0, 0, 0, 5, 5, 5, 5, 0, 0, 0, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6])

In [ ]:
# attention_masks : 문장이 토크나이즈 된 것 중에서 패딩이 아닌 부분은 1, 패딩인 부분은 0으로 마스킹
attention_masks = np.array([[int(i != tokenizer.convert_tokens_to_ids("[PAD]")) for i in ii] for ii in input_ids])

In [ ]:
attention_masks[33]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0])

In [ ]:
# train 데이터에서 10% 만큼을 validation 데이터로 분리
tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids, tags,
                                                            random_state=2018, test_size=0.1)

In [ ]:
tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=0.1)

In [ ]:
# TPU 작동을 위해 실행
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.41.176.162:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.41.176.162:8470


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


In [ ]:
SEQ_LEN = max_len
def create_model():
  model = TFBertModel.from_pretrained("bert-base-multilingual-cased", from_pt=True, num_labels=len(label_dict), output_attentions = False,
    output_hidden_states = False)

  token_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_word_ids') # 토큰 인풋
  mask_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_masks') # 마스크 인풋

  bert_outputs = model([token_inputs, mask_inputs])
  bert_outputs = bert_outputs[0] # shape : (Batch_size, max_len, 30(개체의 총 개수))
  nr = tf.keras.layers.Dense(30, activation='softmax')(bert_outputs) # shape : (Batch_size, max_len, 30)
  
  nr_model = tf.keras.Model([token_inputs, mask_inputs], nr)
  
  nr_model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.00002), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
      metrics=['sparse_categorical_accuracy'])
  nr_model.summary()
  return nr_model

In [ ]:
strategy = tf.distribute.experimental.TPUStrategy(resolver)
# TPU를 활용하기 위해 context로 묶어주기
with strategy.scope():
  nr_model = create_model()
  nr_model.fit([tr_inputs, tr_masks], tr_tags, validation_data=([val_inputs, val_masks], val_tags), epochs=3, shuffle=False, batch_size=bs)

INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Workers: 1


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)
loading configuration file https://huggingface.co/bert-base-multilingual-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/6c4a5d81a58c9791cdf76a09bce1b5abfb9cf958aebada51200f4515403e5d08.0fe59f3f4f1335dadeb4bce8b8146199d9083512b50d07323c1c319f96df450c
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LA

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_word_ids (InputLayer)    [(None, 50)]         0           []                               
                                                                                                  
 input_masks (InputLayer)       [(None, 50)]         0           []                               
                                                                                                  
 tf_bert_model_1 (TFBertModel)  TFBaseModelOutputWi  177853440   ['input_word_ids[0][0]',         
                                thPoolingAndCrossAt               'input_masks[0][0]']            
                                tentions(last_hidde                                               
                                n_state=(None, 50,                                          

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/3


1125/1125 [==============================] - 157s 81ms/step - loss: 0.0590 - sparse_categorical_accuracy: 0.9820 - val_loss: 0.0022 - val_sparse_categorical_accuracy: 0.9991
Epoch 2/3
1125/1125 [==============================] - 86s 76ms/step - loss: 0.0032 - sparse_categorical_accuracy: 0.9988 - val_loss: 0.0012 - val_sparse_categorical_accuracy: 0.9995
Epoch 3/3
1125/1125 [==============================] - 86s 76ms/step - loss: 0.0021 - sparse_categorical_accuracy: 0.9992 - val_loss: 0.0011 - val_sparse_categorical_accuracy: 0.9995


In [60]:
nr_model.save('/content/drive/MyDrive/Colab_Notebooks/m10_딥러닝응용/dataset')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab_Notebooks/m10_딥러닝응용/dataset/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab_Notebooks/m10_딥러닝응용/dataset/assets


UnimplementedError: ignored

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

In [ ]:
y_predicted = nr_model.predict([val_inputs, val_masks])

In [ ]:
f_label = [i for i, j in label_dict.items()]
val_tags_l = [index_to_ner[x] for x in np.ravel(val_tags).astype(int).tolist()]
y_predicted_l = [index_to_ner[x] for x in np.ravel(np.argmax(y_predicted, axis=2)).astype(int).tolist()]
f_label.remove("[PAD]")

In [ ]:

print(classification_report(val_tags_l, y_predicted_l, labels=f_label))

              precision    recall  f1-score   support

           O       1.00      1.00      1.00     53946
          금액       1.00      1.00      1.00      7543
          당도       1.00      1.00      1.00      9184
         바디감       0.99      1.00      1.00      9031
          산미       1.00      1.00      1.00      8420
          종류       1.00      1.00      1.00      6934

   micro avg       1.00      1.00      1.00     95058
   macro avg       1.00      1.00      1.00     95058
weighted avg       1.00      1.00      1.00     95058



In [ ]:
def ner_inference(test_sentence):
  

  tokenized_sentence = np.array([tokenizer.encode(test_sentence, max_length=max_len, truncation=True, padding='max_length')])
  tokenized_mask = np.array([[int(x!=1) for x in tokenized_sentence[0].tolist()]])
  ans = nr_model.predict([tokenized_sentence, tokenized_mask])
  ans = np.argmax(ans, axis=2)

  tokens = tokenizer.convert_ids_to_tokens(tokenized_sentence[0])
  new_tokens, new_labels = [], []
  for token, label_idx in zip(tokens, ans[0]):
    if (token.startswith("##")):
      new_labels.append(index_to_ner[label_idx])
      new_tokens.append(token[2:])
    elif (token=='[CLS]'):
      pass
    elif (token=='[SEP]'):
      pass
    elif (token=='[PAD]'):
      pass
    elif (token != '[CLS]' or token != '[SEP]'):
      new_tokens.append(token)
      new_labels.append(index_to_ner[label_idx])

  for token, label in zip(new_tokens, new_labels):
      print("{}\t{}".format(label, token))

In [ ]:
ner_inference("달달한 레드와인 10만원으로 추천해줄래?")

당도	달
당도	달
당도	한
종류	레
종류	드
O	와
O	인
금액	10
금액	만
금액	원으로
O	추
O	천
O	해
O	줄
O	래
O	?
